In [1]:
%pip install -U langchain-community faiss-cpu langchain-huggingface pymupdf tiktoken langchain-ollama python-dotenv

  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\91939\Downloads\newsai\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install pandas

You should consider upgrading via the 'c:\Users\91939\Downloads\newsai\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
news_data = pd.read_csv(r"C:\Users\91939\Downloads\Cleaned_news.csv")
news_texts = news_data['News'].dropna().tolist()


In [4]:
news_texts[0]

"Lucknow: Three CRPF personnel and the driver of an SUV in the security fleet of Uttar Pradesh Industrial Development Minister Nand Gopal Gupta ‘Nandi' were injured after the SUV, part of the convoy, collided with a tractor near Kanti Chowki in Sant Kabir Nagar on Saturday night. The minister, who was returning from the 35th foundation day ceremony of the Gorakhpur Industrial Development Authority, narrowly escaped the incident, as his vehicle passed moments before the tractor collided with the convoy vehicle. Nandi acted promptly, transporting the injured personnel to Sri Krishna Mission Hospital in Basti in his vehicle. Medical reports revealed head injuries for two CRPF personnel and arm injuries for another. Driver Neeraj also sustained minor injuries. After initial treatment, the injured were transferred to Medanta Hospital in Lucknow for further care."

In [5]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

In [6]:
documents = [Document(page_content=text) for text in news_texts]
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
news_chunks = text_splitter.split_documents(documents)

In [8]:
news_chunks

[Document(metadata={}, page_content="Lucknow: Three CRPF personnel and the driver of an SUV in the security fleet of Uttar Pradesh Industrial Development Minister Nand Gopal Gupta ‘Nandi' were injured after the SUV, part of the convoy, collided with a tractor near Kanti Chowki in Sant Kabir Nagar on Saturday night. The minister, who was returning from the 35th foundation day ceremony of the Gorakhpur Industrial Development Authority, narrowly escaped the incident, as his vehicle passed moments before the tractor collided with the convoy vehicle. Nandi acted promptly, transporting the injured personnel to Sri Krishna Mission Hospital in Basti in his vehicle. Medical reports revealed head injuries for two CRPF personnel and arm injuries for another. Driver Neeraj also sustained minor injuries. After initial treatment, the injured were transferred to Medanta Hospital in Lucknow for further care."),
 Document(metadata={}, page_content='Lucknow: The Uttar Pradesh govt bagged the "Top Achiev

In [9]:
len(news_chunks)

231

In [7]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS 
from langchain_community.docstore.in_memory import InMemoryDocstore

In [8]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')

vector_store = FAISS.from_documents(documents=news_chunks, embedding=embeddings)

# vector_store = FAISS.load_local(news_chunks, embeddings, allow_dangerous_deserialization=True)

In [15]:
question = "who is Nandi?"
docs = vector_store.search(query=question, k=5, search_type="similarity")

In [16]:
docs

[Document(id='0d34e290-50c5-4f12-aadd-4f006d4be676', metadata={}, page_content='It is believed that the bungalow is unlucky for its occupant (inset) Nandi’s nameplate LUCKNOW: Cabinet minister Nand Gopal Gupta Nandi has decided to do something many politicians have not had the courage to. The minister for stamp and registration and civil aviation in the Yogi Aditya Nath government has decided to challenge a long-standing superstition and shift into bungalow number 6 on Kalidas Marg. What makes the act a daring one is that the bungalow, next to the CM’s residence, is considered “haunted”. Poll Do you think the Delhi govt has done enough to improve public healthcare facilities? No Yes It is believed that the occupant of this huge but “unlucky” bungalow sees a decline in both personal and political life.  Now From SP leaders Amar Singh, Rajendra Chaudhary and Javed Abdi to BSP’s Babu Singh Kushwaha and BJP’s SP Singh Baghel, political leaders have had to leave the bungalow after suffering

In [17]:

retriever = vector_store.as_retriever(search_type = 'similarity', 
                                      search_kwargs = {'k': 3})

retriever.invoke(question)

[Document(id='0d34e290-50c5-4f12-aadd-4f006d4be676', metadata={}, page_content='It is believed that the bungalow is unlucky for its occupant (inset) Nandi’s nameplate LUCKNOW: Cabinet minister Nand Gopal Gupta Nandi has decided to do something many politicians have not had the courage to. The minister for stamp and registration and civil aviation in the Yogi Aditya Nath government has decided to challenge a long-standing superstition and shift into bungalow number 6 on Kalidas Marg. What makes the act a daring one is that the bungalow, next to the CM’s residence, is considered “haunted”. Poll Do you think the Delhi govt has done enough to improve public healthcare facilities? No Yes It is believed that the occupant of this huge but “unlucky” bungalow sees a decline in both personal and political life.  Now From SP leaders Amar Singh, Rajendra Chaudhary and Javed Abdi to BSP’s Babu Singh Kushwaha and BJP’s SP Singh Baghel, political leaders have had to leave the bungalow after suffering

In [10]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain import hub

In [11]:
prompt = hub.pull("rlm/rag-prompt")
prompt

c:\Users\91939\Downloads\newsai\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [12]:
prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [13]:
llm = ChatOllama(model='mistral', base_url='http://localhost:11434')
llm.invoke('hi')

AIMessage(content=" Hello! How can I help you today? Is there something specific you would like to discuss or learn about?\n\nFor example, if you have a question about technology, science, art, literature, or anything else, feel free to ask and I'll do my best to provide an informative response. If you're feeling a bit down, we can talk about that too. Or maybe you just want to chat casually - I'm here for that as well! Let me know what's on your mind. :)", additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-01-05T12:21:34.2521072Z', 'done': True, 'done_reason': 'stop', 'total_duration': 31396581500, 'load_duration': 8812757000, 'prompt_eval_count': 6, 'prompt_eval_duration': 3408000000, 'eval_count': 110, 'eval_duration': 19162000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-e4ad1d7c-59a1-41f8-ad91-60087453c263-0', usage_metadata={'input_tokens': 6, 'output_tokens': 110, 'total_tokens': 116})

In [18]:

def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

context = format_docs(docs)
print(context)

It is believed that the bungalow is unlucky for its occupant (inset) Nandi’s nameplate LUCKNOW: Cabinet minister Nand Gopal Gupta Nandi has decided to do something many politicians have not had the courage to. The minister for stamp and registration and civil aviation in the Yogi Aditya Nath government has decided to challenge a long-standing superstition and shift into bungalow number 6 on Kalidas Marg. What makes the act a daring one is that the bungalow, next to the CM’s residence, is considered “haunted”. Poll Do you think the Delhi govt has done enough to improve public healthcare facilities? No Yes It is believed that the occupant of this huge but “unlucky” bungalow sees a decline in both personal and political life.  Now From SP leaders Amar Singh, Rajendra Chaudhary and Javed Abdi to BSP’s Babu Singh Kushwaha and BJP’s SP Singh Baghel, political leaders have had to leave the bungalow after suffering a blow to their careers. The young minister Nandi, however, is unfazed and read

In [19]:

rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:

question = "Who is Nand gopal gupta?,give brief summary about him?"
response = rag_chain.invoke(question)

print(response)

 * Nand Gopal Gupta, also known as "Nandi", is a cabinet minister in Uttar Pradesh.
   * He holds the portfolios of industrial development, export promotion, NRI, and investment promotion.
   * He was involved in a controversy over the joining of SP leader Raees Chandra Shukla into BJP, which he considered an insult and a serious issue.
   * Nandi alleged that this move was illegal, against democratic values, and a deep conspiracy.
   * He is known for his aggressive stance and has made controversial remarks in the past, such as comparing BSP's national president Mayawati with Ramayana character Shurpanakha.
   * In a different context, Nandi was also involved in efforts to review the progress of MoUs signed during the UP Global Investors Summit, aiming to lay foundation stones for an investment proposal of Rs 10 lakh crore.


In [26]:

question = "give me the positive impressions on Nandi?"
response = rag_chain.invoke(question)

print(response)

 * Nand Gopal Gupta Nandi is a cabinet minister in the Yogi Adityanath government of Uttar Pradesh.
   * He has decided to shift into bungalow number 6 on Kalidas Marg, which is considered "haunted".
   * Nandi believes he is blessed by the divine and has taken remedial measures like installing a wind mill and performing a puja to make the bungalow perfect.
   * He apprised representatives of state governments about upcoming policy framework in Uttar Pradesh during the national export policy consultation led by Union minister Piyush Goyal.
   * The eighth and final domestic roadshow held in Chandigarh ahead of the UP Global Investors' Summit 2023 received business and investment proposals worth over Rs 10,000 crore, with Nandi leading the roadshow.
   * He stated that Uttar Pradesh has set a target to increase its exports to Rs 3 lakh crore over the next three years.
   * Under the leadership of Chief Minister Yogi Adityanath, Uttar Pradesh is emerging as the 'New Uttar Pradesh of New 

In [27]:
question = "give me the Negative impressions on Nandi?"
response = rag_chain.invoke(question)

print(response)

 * The bungalow that Nandi is moving into is considered "unlucky" or "haunted".
   * Many politicians have left this bungalow after suffering a decline in their careers.
   * Nandi has heard of the superstitions surrounding the bungalow, but he is not deterred and has taken remedial measures to make it auspicious.
   * Nandi's shift into the bungalow has been met with questions about its alleged unluckiness.
   * Nandi believes he is blessed by the divine and is prepared to move into the bungalow despite its reputation.
   * It is not clear from the context whether there are any specific negative impressions of Nandi himself, only that the bungalow he is moving into has a negative reputation.


In [28]:
question = "Give me an overview of the a Nandi in his political career?"
response = rag_chain.invoke(question)

print(response)

 - Nand Gopal Gupta, commonly known as 'Nandi', is a politician currently contesting the Allahabad South assembly segment for the BJP.
   - He made his electoral debut in 2007 and caused the BJP to lose its grip over the constituency that it had never lost since 1989.
   - Nandi defeated Keshri Nath Tripathi, a five-time MLA and former UP assembly Speaker who was also the BJP's state unit president at that time.
   - He was expelled from the BSP three years ago but was recently inducted into the BJP last month, causing dissatisfaction among some members, including Allahabad MP Shyama Charan Gupta.
   - Nandi is known for his dynamism and ability to marshal resources.
   - He has been embroiled in controversy over allegations of making inflammatory comments against political opponents, with complaints filed against him by the BSP and a social activist.


In [20]:
question = "What is the interesting news happening in Lucknow?"
response = rag_chain.invoke(question)

print(response)

1. The first roadshow for the UP Global Investors' Summit was held in Lucknow on Wednesday, receiving investment proposals of around Rs 76,867 crore from investors both in India and abroad.
2. At least 79 MoUs and expressions of interest were signed during this event.
3. The summit is scheduled to begin on February 10 and is expected to be attended by more than 10,000 people.
4. Notable dignitaries including President Droupadi Murmu and Prime Minister Narendra Modi are expected to attend the summit.
5. Arrangements for these VIP guests and investors, including facilities like movement, seating, and others, have been made at the venue.
6. A food court has been set up adjacent to one of the blocks, along with a VIP lounge and lounges for partner countries, industry, and knowledge partners.
7. The fourth block is being prepared for pre-function activities, the fifth block is for cultural activities, and the sixth block is being prepared for B2B meetings where investment proposals and MoUs

In [21]:
question = "List of the positive keywords associated with Nandi and explain each in brief?"
response = rag_chain.invoke(question)

print(response)

1. "New Uttar Pradesh of New India" - This phrase indicates a positive change and development in the state under the current leadership.

2. "Largest consumer state in India" - This implies that UP has a vast market potential due to its large population.

3. "Availability of a large, young and skilled workforce" - This suggests that there is an abundance of manpower with potential skills in the state.

4. "Ease of doing business" - This term indicates that the government is making efforts to facilitate and encourage businesses.

5. "Transparent work culture" - This phrase suggests that the government is working honestly and openly, which could attract investors.

6. "Balance between tradition and modernity" - This implies that UP is able to preserve its cultural heritage while also embracing modernization and development.

7. "More than 20,000 employment opportunities" - This suggests that the state's economic growth will lead to job creation.

8. "Investment proposals of over Rs 10,00

In [22]:
question = "List of the Negative keywords associated with Nandi and explain each in brief?"
response = rag_chain.invoke(question)

print(response)

 * "Haunted": This term is used in the context of a bungalow that is believed to be unlucky for its occupant and associated with a decline in both personal and political life. The minister Nand Gopal Gupta 'Nandi' has chosen to move into this bungalow despite these superstitions.
   * "Manipulate": This term is used when the minister Nandi accused SP chief Akhilesh Yadav of manipulating the people of Azamgarh in the wake of bye-elections. He claimed that Akhilesh had made false promises during the 2019 Lok Sabha elections and then betrayed the mandate by resigning mid-term.
   * "Opportunistic": This term is used to describe Akhilesh Yadav's behavior, as Nandi stated that he turns up at weddings and condolence meetings before the bye-election, which suggests that his actions are motivated by political gain rather than sincerity.


In [23]:
question = "what can Nandi do, to improve campaign outreach?"
response = rag_chain.invoke(question)

print(response)

1. Apologize for any offensive comments made during election rallies, as shown by Nandi's apology to Mayawati.

2. Engage with local workers and leaders in regular organizational meetings, such as the tiffin meeting held in Prayagraj.

3. Share success stories and accomplishments achieved under the leadership of Prime Minister Narendra Modi to motivate and inform the public.

4. Prioritize promoting industries within the state, like wood industry, as demonstrated by Nandi's meeting with key stakeholders for this purpose.

5. Listen to concerns and issues faced by industry players and address them accordingly, as shown in the meeting between senior officials, cabinet minister Nandi, and stakeholders of the wood industry.

6. Encourage agro-forestry to increase farmers' income, as suggested by private industry stakeholders during the meeting with Nandi.

7. Consider the impact of pending issues before the Supreme Court, like the licensing of private players interested in opening MDF and 